In [ ]:
from functions import *
from parameters import *

from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
import torch
import torch.nn.functional as F

from statsmodels.nonparametric.smoothers_lowess import lowess

from tqdm import tqdm
from IPython.display import display, HTML

The raw data, as a CSV file.

In [ ]:
LOG( "Data (data-frame)" )
filename = "raw/data_ml.csv"
LOG( f"Reading {filename} [20 seconds]" )
d = pd.read_csv(filename)
d['date'] = pd.to_datetime( d['date'] )

predictors = list( signs.keys() )
target = 'R1M_Usd'

Rather than a data-frame, it is often easier to have a list of matrices, one per variable, 
with one row per stock and one column per date: we can easily combine them or apply some function to each row, or each column.

In [ ]:
LOG( "Data (list of matrices)" )
LOG( "Read data/data_ml.pickle" )
dd = load( "data/data_ml.pickle" )

# Single signals
For any of the input variables, we can divide the universe into quintiles, long the top quintile, and short the bottom quintile.

In [ ]:
LOG( "Backtest a single signal" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = predictors[0]
LOG( f"  {signal}" )
x = dd[signal].copy() * signs[signal]
x.fillna(.5, inplace=True)  ## Replace missing values with the median (0.5, since the predictors are uniform)
x = np.where( dd['universe'], 1, np.nan ) * x   # Only invest in stocks in the universe
r = signal_backtest(x, y, date = DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.set_title(signal if signs[signal]>=0 else f"- {signal}")
ax.text(0.02, .97, f"μ={100*r['performance'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*r['performance'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={r['performance'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig(f'plots/signal_wealth_{signal}.pdf')
plt.show()

r['performance']

# Lasso
We try to forecast the 1-month forward return from all the input variables. 

As we let the scale of the $L^1$ penalty vary, we have a family of increasingly complex models, from the intercept-only one, to the (unpenalized) least-squares model.

In [ ]:
LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Lasso" )
alphas, coef, _ = lasso_path( X = x, y = y, max_iter = 10_000 )

In [ ]:
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

LOG( "Loop: backtests of the lasso models [5 minutes]" )
res = {}
for j in tqdm(range(len(alphas))):
    signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
    for i,predictor in enumerate(predictors):
        signal += coef[i,j] * dd[predictor].fillna(.5)
    signal = np.where( dd['universe'], 1, np.nan ) * signal
    r = signal_backtest(signal, y, date=DATE1)
    r['performance'  ]['alpha'] = alphas[j]
    r['in-sample'    ]['alpha'] = alphas[j]
    r['out-of-sample']['alpha'] = alphas[j]
    r['performance'  ]['period'] = 'all'
    r['in-sample'    ]['period'] = 'in-sample'
    r['out-of-sample']['period'] = 'out-of-sample'    
    res[ f"{alphas[j]} all" ] = r['performance']
    res[ f"{alphas[j]} in"  ] = r['in-sample']
    res[ f"{alphas[j]} out" ] = r['out-of-sample']
    
    if j == 20:
        # Wealth curves
        fig, ax = plt.subplots()
        for i in range(6):
            ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
        ax.set_yscale('log')
        ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )        
        ax.set_title( f"alpha={alphas[j]:5.3}")
        ax.text(0.02, .97, f"μ={100*r['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
        ax.text(0.02, .90, f"σ={100*r['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
        ax.text(0.02, .83, f"IR={r['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
        fig.savefig(f"plots/lasso_wealth_{j}.pdf")
        plt.show()
        display( r['out-of-sample'] )
        
res = pd.concat( res.values() )

In [ ]:
def plot_lasso(coefs, n=20, ax=None):
    assert isinstance( coefs, pd.DataFrame ), f"Expecting a DataFrame of coefficients, with one row per predictor, got a {type(coefs)}"

    ## Reorder the coefficients
    def f(u):
        v = np.argwhere(u != 0)
        v = v.flatten()
        if len(v) == 0:
            return len(u)
        return v[0]
    i = np.apply_along_axis( f, 1, coefs )
    i = np.argsort(i)
    coefs = coefs.iloc[i,:]

    import copy
    cmap = copy.copy(matplotlib.cm.get_cmap("RdBu"))    
    cmap.set_bad('white')    # Plot the zeros as "white", not "grey" (after replacing them with nan)

    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots(figsize=(10,5))
    tmp = coefs.iloc[:n,:].copy()
    m = np.max(np.abs(tmp.values))
    tmp[ tmp == 0 ] = np.nan
    ax.imshow(tmp, vmin=-m, vmax=m, cmap=cmap, aspect='auto', interpolation='none')
    ax.set_yticks(range(tmp.shape[0]))
    ax.set_yticklabels( tmp.index )
    ax.axes.yaxis.set_ticks_position("right")
    ax.spines['left'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.axes.xaxis.set_visible(False)
    if ax_was_None:
        fig.tight_layout()
        plt.show()   
        
fig, ax = plt.subplots(figsize=(8,5))
plot_lasso(pd.DataFrame( coef, index=predictors ), ax = ax)
fig.tight_layout()
fig.savefig("plots/lasso_coefs.pdf")
plt.show()

We expect the in-sample performance to increase, and the out-of-sample performance to increase and then decrease -- 
this is not what we see here: the out-of-sample performance does not decrease, and a linear model performs extremely well.

In [ ]:
i1 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'in-sample' )
i2 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'out-of-sample' )
i3 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'all' )
fig, axs = plt.subplots(1,3, figsize=(15,5))
for j,key in enumerate(['Information Ratio', 'CAGR', 'Annualized Volatility']):
    ax = axs[j]
    ax.plot( res[i1]['alpha'], res[i1][key], label = "in-sample" )
    ax.plot( res[i2]['alpha'], res[i2][key], label = "out-of-sample" )
    #ax.plot( res[i3]['alpha'], res[i3][key], label = "all" )
    ax.set_xlabel('Alpha')
    ax.set_xscale('log')
    ax.invert_xaxis()
    ax.set_title(key)
    ax.legend()
fig.savefig("plots/lasso_regularization_path.pdf")
plt.show()

number_of_predictors = ( coef != 0 ).sum(axis=0)
fig, ax = plt.subplots()
ax.plot( alphas, number_of_predictors )
ax.set_xlabel('Alpha')
ax.set_xscale('log')
ax.invert_xaxis()
ax.set_title('Number of predictors')
fig.savefig("plots/lasso_number_of_predictors.pdf")
plt.show()

# Constrained regression

To improve the performance of the linear models, to prevent it from overfitting the data, 
and to ensure the model remains interpretable, we can add constraints on the signs of the coefficients of the regression.
Indeed we know in advance whether each predictor has a positive or negative impact on future returns:
for instance, volatility has a negative impact, while earnings yield has a positive impact.

In [ ]:
from scipy.optimize import lsq_linear

LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Fit the model" )
r = lsq_linear(
    x, y,
    (
        [ 0      if signs[u] > 0 else -np.inf for u in predictors ],
        [ np.inf if signs[u] > 0 else 0       for u in predictors ],
    )
)

# It is not supposed to be that sparse: usually, 2/3 of the coefficients are non-zero...
w = np.round( 1e6 * r.x )
{ p: w[i] for i,p in enumerate(predictors) if w[i] != 0 }

In [ ]:
LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += r.x[i] * dd[predictor].fillna(.5)
    #signal += r_rev.x[i] * dd[predictor].fillna(.5)
    #signal += r_ols.x[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Constrained regression')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/constrained_regression_wealth.pdf")
plt.show()

res['out-of-sample']

# Deep learning: linear model
This is equivalent to the models above, but implemented with a neural network.
* Linear model
* Change the loss function
* Add mini-batches
* Add sign constraints

### Linear model; forecast the returns

In [ ]:
## Data: as above 

LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Model" )

class Linear1(torch.nn.Module):
    def __init__(self,k):
        super(Linear1,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,x):
        y = self.linear(x)
        return y

x = torch.tensor(x.values,               dtype=torch.float32)
y = torch.tensor(y.values.reshape(-1,1), dtype=torch.float32)

model = Linear1(x.shape[1])
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

LOG( "Loop [2 minutes]" )
for t in tqdm(range(5000)):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

beta = list( model.parameters() )[0]
beta = beta.detach().numpy().flatten()

r = None

LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += beta[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Liner model (via pytorch)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model1_linear_wealth.pdf")
plt.show()

res['out-of-sample']

### Linear model; data as id×signal×date array; forecast the returns

Same model as above, but the data is no longer in an (id,date)×signal (2-dimensional) table, but in an id×date×signal 3-dimensional array.

The model and the performance are similar but, for some reason, fitting the model is much more time-consuming.

In [ ]:
LOG( "Training data (3-dimensional array)" )

def get_data_3(all=False, flip_signs=False):
    """
    Data, as tensors.
    Arguments: all: whether ti return all the data or just the training data
    Returns:   x: id×date×signal
               y: id×date, forward ratio returns
               universe: id×date, 0 or 1
    Global variables used: d, target, predictors, DATE1
    """

    train = data_frame_to_list(d, id_name = 'stock_id', date_name = 'date')
    if all:
        i = np.array( [ True for u in train[ predictors[0] ].columns ] )
    else:
        i = np.array([ str(u) < DATE1 for u in train[ predictors[0] ].columns ])
    train = { k: v.T[i].T for k,v in train.items() }
        #x = train[ predictors ]
        #y = np.log1p(train[ target ])
    y = train[target]
    if flip_signs: 
        x = [ signs[k] * train[k] for k in predictors ]
    else:
        x = [ train[k] for k in predictors ]
    y = y.values
    x = np.stack( [ u.values for u in x ] )

    y = np.log1p(y)
    universe = np.where( np.isfinite(y), 1, 0 )
    # x = x.fillna(.5)
    x = np.nan_to_num(x, nan=.5)
    y = np.nan_to_num(y, nan=0)

    n = y.shape[0]   # Number of stocks
    l = y.shape[1]   # Number of dates
    k = x.shape[0]   # Number of predictors
    assert k == len(predictors)
    assert n == x.shape[1]
    assert l == x.shape[2]

    # x is now: signal×id×date

    x = x.transpose([1,2,0])
    assert x.shape == (n,l,k) # id×date×signal
    assert y.shape == (n,l)   # id×date
    assert universe.shape == (n,l)

    if False: 
        x = x[:5,:4,:3]
        y = y[:5,:4]
        universe = universe = universe[:5,:4]

    assert ( ~ np.isfinite(x) ).sum() == 0
    assert ( ~ np.isfinite(y) ).sum() == 0
    
    return x, y, universe

x, y, universe = get_data_3()

In [ ]:
class Linear2(torch.nn.Module):
    def __init__(self,k):
        super(Linear2,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,x):
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        y = y[:,:,0]         #  n×l
        return y

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

In [ ]:
model = Linear2(x.shape[2])
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

LOG( "Loop [LONG: 20 minutes for 5000 epochs]" )
N = 5000
losses = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    y_pred = model(x) * universe[:,:,0]
    loss = criterion(y_pred,y[:,:,0])
    losses[t] = loss.item()
    pbar.set_description( f"Loss={loss.item():.5f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig, ax = plt.subplots()
ax.plot( losses )
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_xscale('log')
fig.savefig("plots/model2_linear_3d_loss.pdf")
plt.show()

In [ ]:
# Backtest the resulting strategy (exactly the same code as above)

beta = list( model.parameters() )[0]
beta = beta.detach().numpy().flatten()

r = None

LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += beta[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Linear model (via pytorch)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model2_linear_3d_wealth.pdf")
plt.show()

res['out-of-sample']

### Simple, nonlinear model

This does not really work: the optimization often remains stuck. 
Restarting the optimization several (10?) times eventually gives something acceptable.

In [ ]:
class NonLinear2(torch.nn.Module):
    def __init__(self,k):
        super(NonLinear2,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,x):
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = F.relu(y)
        y = self.fc2(y); y = F.relu(y)
        y = self.fc3(y); # n×l×1
        y = y[:,:,0]         #  n×l
        return y

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = NonLinear2(x.shape[2])
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

LOG( "Loop [LONG: 20 minutes for 5000 epochs]" )
N = 5000
losses = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    y_pred = model(x) * universe[:,:,0]
    loss = criterion(y_pred,y[:,:,0])
    losses[t] = loss.item()
    pbar.set_description( f"Loss={loss.item():.5f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig, ax = plt.subplots()
ax.plot( losses )
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_xscale('log')
fig.savefig("plots/model2_nonlinear_3d_loss.pdf")
plt.show()

In [ ]:
x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model(x).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Non-linear')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model2_nonlinear_3d_wealth.pdf")
plt.show()

res['out-of-sample']

### Compute portfolio weights; optimize the information ratio

In [ ]:
x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

In [ ]:
class Linear3(torch.nn.Module):
    def __init__(self,k):
        super(Linear3,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        p = y.exp()             # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p
    
model = Linear3(x.shape[2])
# model( (x,universe) ).detach().numpy().sum(axis=0)   # Should sum to 1 for each date

In [ ]:
## Loop to maximize the IR

LOG( "[LONG] 50 minutes for 10,000 epochs" )

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    w = model( (x,universe) )
    ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={IR.item():.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig, ax = plt.subplots()
ax.plot( IRs )
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model3_linear_IR_loss.pdf")
plt.show()

In [ ]:
x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model3_linear_IR_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back
s = analyze_returns( r[ r.index > DATE1 ], as_df = True )

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights)')
ax.text(0.02, .97, f"μ={100*s.iloc[0,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*s.iloc[0,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={s.iloc[0,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model3_linear_IR_wealth_weights.pdf")
plt.show()

s

### Mini-batches
To make the model more robust, we train it on "mini-batches", defined by a random subset of stocks, and a time interval (contiguous dates).
These are not real mini-batches, because the loss function is not a sum over the observations

In [ ]:
LOG( "[LONG] 30 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear3(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model3b_linear_IR_minibatches_loss.pdf")
plt.show()

In [ ]:
## Wealth curves

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (signal, mini-batches)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model3b_linear_IR_minibatches_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back
s = analyze_returns( r[ r.index > DATE1 ], as_df = True )

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights, mini-batches)')
ax.text(0.02, .97, f"μ={100*s.iloc[0,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*s.iloc[0,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={s.iloc[0,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model3b_linear_IR_minibatches_wealth_weights.pdf")
plt.show()

s

### Long-short strategy

In [ ]:
class Linear4(torch.nn.Module):
    def __init__(self,k):
        super(Linear4,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        y = y * universe
        long  = torch.where( y > 0,  y, torch.zeros(1) )
        short = torch.where( y < 0, -y, torch.zeros(1) )
        long  = long [:,:,0]  #  n×l
        short = short[:,:,0]
        long  = long  / ( 1e-6 +  long.sum(axis=0) )
        short = short / ( 1e-6 + short.sum(axis=0) )
        p = long - short
        return p

In [ ]:
## Same code as above

LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear4(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole training sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model4_linear_IR_LS_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (long-short, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model4_linear_IR_LS_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back
s = analyze_returns( r[ r.index > DATE1 ], as_df = True )   # Very bad...

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (long-short, weights)')
ax.text(0.02, .97, f"μ={100*s.iloc[0,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*s.iloc[0,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={s.iloc[0,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model4_linear_IR_LS_wealth_weights.pdf")
plt.show()

s

### TODO: Add transaction costs

In [ ]:
## TODO

# Deep learning: nonlinear model

### Several linear layers

In [ ]:
class Linear6(torch.nn.Module):
    def __init__(self,k):
        super(Linear6,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x)
        y = self.fc2(y)
        y = self.fc3(y)      # n×l×1
        p = y.exp()          # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear6(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model6_linear_IR_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (deep linear, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model6_linear_IR_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back
s = analyze_returns( r[ r.index > DATE1 ], as_df = True )   

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (deep linear, weights)')
ax.text(0.02, .97, f"μ={100*s.iloc[0,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*s.iloc[0,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={s.iloc[0,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model6_linear_IR_wealth_weights.pdf")
plt.show()

s

### Nonlinear

In [ ]:
class NonLinear6(torch.nn.Module):
    def __init__(self,k):
        super(NonLinear6,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = F.relu(y)
        y = self.fc2(y); y = F.relu(y)
        y = self.fc3(y)      # n×l×1
        p = y.exp()          # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = NonLinear6(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model6_nonlinear_IR_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (non-linear, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model6_nonlinear_IR_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back
s = analyze_returns( r[ r.index > DATE1 ], as_df = True )   

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights)')
ax.text(0.02, .97, f"μ={100*s.iloc[0,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*s.iloc[0,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={s.iloc[0,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model6_nonlinear_IR_wealth_weights.pdf")
plt.show()

s

In [ ]:
# Try to understand what the model actually learned

def plot_copula(x,y, ax=None, title=None, unif=True, cmap='Blues'):
    i = np.isfinite(x) & np.isfinite(y)
    x = x[i]
    y = y[i]

    if unif:
        x = uniformize(x)
        y = uniformize(y)
        xmin, xmax, ymin, ymax = 0,1, 0,1
    else:
        xmin, xmax = min(x), max(x)
        ymin, ymax = min(y), max(y)

    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots( figsize = (4,4) )

    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x,y])
    kernel = scipy.stats.gaussian_kde(values)
    f = np.reshape( kernel(positions).T, xx.shape )
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    cfset = ax.contourf(xx, yy, f, cmap=cmap)
    cset = ax.contour(xx, yy, f, colors='k')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    if title is not None:
        ax.set_title(title)
    if ax_was_None:
        plt.show()
        
def remove_empty_axes(axs):
    for ax in axs.flatten():
        if (not ax.lines) and (not ax.collections) and (not ax.has_data()):
            ax.axis('off')
        

In [ ]:
LOG( "Copula densities [VERY LONG: 1.5 hours]")        
nr, nc = mfrow(x.shape[2], aspect=1)
fig, axs = plt.subplots( nr, nc, figsize=(29.7/1.5,21/1.5) )
for i in tqdm(range(x.shape[2])):
    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    b = signal.values
    b = np.apply_along_axis( uniformize, 0, b )    
    ax = axs.flatten()[i]
    plot_copula(a.flatten(),b.flatten(), ax = ax)
    ax.set_title( predictors[i] )
remove_empty_axes(axs)
fig.tight_layout()
fig.subplots_adjust(hspace=.2, wspace=.05)
fig.savefig('plots/model6_nonlinear_copulas_all.pdf')
plt.show()

In [ ]:
# Individual plots
for i in tqdm(range(x.shape[2])):
    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    b = signal.values
    b = np.apply_along_axis( uniformize, 0, b )    
    fig, ax = plt.subplots()
    plot_copula(a.flatten(),b.flatten(), ax = ax)
    ax.set_title( predictors[i] )
    fig.tight_layout()
    fig.savefig(f'plots/model6_nonlinear_copulas_{predictors[i]}.pdf')
    plt.close()

In [ ]:
nr, nc = mfrow(x.shape[2], aspect=1)
fig, axs = plt.subplots( nr, nc, figsize=(29.7/1.5,21/1.5) )
for i in tqdm(range(x.shape[2])):
    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    
    if False: 
      # Some of the variables just measure size: normalize them
      # (not really possible with the data we have: we can only see something for FCF -- for most other variables, the effect of the MCap is overpowering)
      if i > 0:
        a = a - x.detach().numpy()[:,:,0]
        a = np.apply_along_axis( uniformize, 0, a )    
        
    b = signal.values.copy()
    a = np.floor( a * 20 * .9999 )
    b = np.apply_along_axis( uniformize, 0, b )
    a = a.flatten()
    b = b.flatten()
    c = pd.DataFrame( { 'quantile': a, 'value': b } )
    c = c.pivot_table(values='value', columns = 'quantile', aggfunc='mean')
    ax = axs.flatten()[i]
    ax.plot( c.columns, c.values.flatten() )
    ax.scatter( c.columns, c.values.flatten() )
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)    
    ax.set_title( predictors[i] )
remove_empty_axes(axs)
fig.tight_layout()
fig.subplots_adjust(hspace=.2, wspace=.05)
fig.savefig('plots/model6_nonlinear_median_per_quantile_all.pdf')
plt.show()

LOG( "Individual plots" )
for i in tqdm(range(x.shape[2])):
   
    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    b = signal.values.copy()
    a = np.floor( a * 20 * .9999 )
    b = np.apply_along_axis( uniformize, 0, b )
    a = a.flatten()
    b = b.flatten()
    c = pd.DataFrame( { 'quantile': a, 'value': b } )
    c = c.pivot_table(values='value', columns = 'quantile', aggfunc='mean')
    fig, ax = plt.subplots()
    ax.plot( c.columns, c.values.flatten() )
    ax.scatter( c.columns, c.values.flatten() )
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)    
    ax.set_title( predictors[i] )
    fig.tight_layout()
    fig.savefig(f'plots/model6_nonlinear_median_per_quantile_{predictors[i]}.pdf')
    plt.close()

In [ ]:
nr, nc = mfrow(x.shape[2])
fig, axs = plt.subplots( nr, nc, figsize=(20,20) )
for i in tqdm(range(x.shape[2])):
    ax = axs.flatten()[i]

    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    b = signal.values.copy()
    a = np.floor( a * 20 * .9999 )
    b = np.apply_along_axis( uniformize, 0, b )
    a = a.flatten()
    b = b.flatten()
    c = pd.DataFrame( { 'quantile': a, 'value': b } )
    c1 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 25))
    c2 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 50))
    c3 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 75))
    c0 = c1.columns
    c1 = c1.values.flatten()
    c2 = c2.values.flatten()
    c3 = c3.values.flatten()
    ax.fill_between( c0, c1, c3, color='lightblue')
    ax.plot(c0, c1, color='tab:blue')
    ax.plot(c0, c3, color='tab:blue')
    ax.plot(c0, c2, marker='o', color='black')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)    
    ax.set_title( predictors[i] )
remove_empty_axes(axs)
fig.tight_layout()
fig.subplots_adjust(hspace=.2, wspace=.05)
fig.savefig('plots/model6_nonlinear_quartiles_per_quantile_all.pdf')
plt.show()

In [ ]:
    a = np.where( universe[:,:,0], x[:,:,i], np.nan )
    b = signal.values.copy()
    a = np.floor( a * 20 * .9999 )
    b = np.apply_along_axis( uniformize, 0, b )
    a = a.flatten()
    b = b.flatten()
    c = pd.DataFrame( { 'quantile': a, 'value': b } )
    c1 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 25))
    c2 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 50))
    c3 = c.pivot_table(values='value', columns = 'quantile', aggfunc=lambda x: np.percentile(x, 75))
    c0 = c1.columns
    c1 = c1.values.flatten()
    c2 = c2.values.flatten()
    c3 = c3.values.flatten()
    fig, ax = plt.subplots()
    ax.fill_between( c0, c1, c3, color='lightblue')
    ax.plot(c0, c1, marker='o', color='tab:blue')
    ax.plot(c0, c3, marker='o', color='tab:blue')
    ax.plot(c0, c2, marker='o', color='black')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)    
    ax.set_title( predictors[i] )
    plt.show()

In [ ]:
for which in ['scatter', 'hexbin']:
    LOG( which )
    nr, nc = mfrow(x.shape[2], aspect=1)
    fig, axs = plt.subplots( nr, nc, figsize=(29.7/1.5,21/1.5) )
    for i in tqdm(range(x.shape[2])):
        a = np.where( universe[:,:,0], x[:,:,i], np.nan )
        b = signal.values.copy()
        b = np.apply_along_axis( uniformize, 0, b )
        a = a.flatten()
        b = b.flatten()
        ax = axs.flatten()[i]
        if which == 'scatter': 
            ax.scatter(
                a + .01 * np.random.uniform(-1,1,n), 
                b + .01 * np.random.uniform(-1,1,n),
                alpha=1/255, 
                s=10
            )
        else: 
            ax.hexbin( a, b, gridsize=20, cmap='Blues' )
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)    
        ax.set_title( predictors[i] )
    remove_empty_axes(axs)
    fig.tight_layout()
    fig.subplots_adjust(hspace=.22, wspace=.05)
    if which == 'scatter':
        LOG( f"{which}: PNG file" )
        fig.savefig(f'plots/model6_nonlinear_{which}_all.png', facecolor='white', transparent=False)  # The PDF file would be too large...
    else: 
        LOG( f"{which}: PDF file" )
        fig.savefig(f'plots/model6_nonlinear_{which}_all.pdf')
    LOG( "Plot" )
    plt.show()
    
    LOG( f"{which}: Individual plots" )
    for i in tqdm(range(x.shape[2])):

        a = np.where( universe[:,:,0], x[:,:,i], np.nan )
        b = signal.values.copy()
        b = np.apply_along_axis( uniformize, 0, b )
        a = a.flatten()
        b = b.flatten()
        if which == 'scatter': 
            fig, ax = plt.subplots(figsize=(19.20,10.80))
        else: 
            fig, ax = plt.subplots()
        if which == 'scatter': 
            n = len(a)
            ax.scatter( 
                a + .01 * np.random.uniform(-1,1,n), 
                b + .01 * np.random.uniform(-1,1,n),
                alpha=1/255, s=200
            )
            ax.set_xlim(0,1)
            ax.set_ylim(0,1)            
        else: 
            ax.hexbin( a, b, gridsize=20, cmap='Blues' )
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)    
        ax.set_title( predictors[i] )        
        fig.tight_layout()
        if which == 'scatter':
            fig.savefig(f'plots/model6_nonlinear_{which}_{predictors[i]}.png', facecolor='white', transparent = False)
        else:
            fig.savefig(f'plots/model6_nonlinear_{which}_{predictors[i]}.pdf')
        plt.close()    

# Deep learning: lattice networks

### Positive weights
Simply constraint the weights to be positive is too restrictive: with ReLU nonlinearities, we end up with increasing *convex* functions.

Convergence issues: nothing usable.
I made the following changes: 
- Use an absolute value instead of a softplus to ensure the weights are positive
- To make the weights positive, replace $\exp y$ with $\text{softplus}(y)$, and add $+10^{-6}$ lest all the weights be too close to zero.

In [ ]:
class LinearPositive(torch.nn.Module):
    def __init__(self, n, k): 
        super(LinearPositive, self).__init__()
        self.layer = torch.nn.Linear(n,k)
    def forward(self,x):
        w = self.weight_actual()
        y = F.linear(x,w) + self.layer.bias
        return y
    def weight_actual(self):
        #return self.layer.weight
        #return torch.nn.Softplus()( self.layer.weight )
        return self.layer.weight.abs()
    
class Calibrator(torch.nn.Module):
    def __init__(self, k=5, xmin=-1, xmax=+1):
        assert k >= 2
        super(Calibrator, self).__init__()
        self.k = k
        self.xs = torch.Tensor( np.linspace(xmin, xmax, k) )
        ys = np.random.uniform(-1,1, size=k)
        ys[0] = xmin
        ys[1] = xmax
        ys = sorted(ys)
        ys = np.diff(ys)
        ys = inverse_softplus(ys)
        self.ys = torch.nn.Parameter( torch.Tensor( ys ) )
    def ys_actual(self):
        ys = self.ys
        ys = F.softplus(ys)
        ys = torch.cumsum(ys,0)
        y = torch.zeros(self.k)
        y[1:] = ys
        y = y - 1
        return y
    def interpolate(self, xs, ys, x_new ):
        ## k points, k-1 intervals between them, k+1 intervals if we include the infinite ones on the left and right
        k = len(xs)
        slopes = ( ys[1:] - ys[:-1] ) / ( xs[1:] - xs[:-1] )
        #print( slopes )
        current_slope = 0
        X = torch.zeros( x_new.shape )
        X = X + ys[0]
        for i in range(k-1): 
            X = X + F.relu( x_new - xs[i] ) * ( slopes[i] - current_slope )
            current_slope = slopes[i]
            #print( current_slope )
        X = X + F.relu( x_new - xs[k-1] ) * ( 0 - current_slope )
        return X
    def interpolate_test(self):
        """Check that my interpolation function works as it should"""
        xs = torch.Tensor( np.linspace(0,1,6) )
        ys = torch.Tensor( np.random.uniform( size=6 ) )
        x_new = torch.Tensor( np.linspace(-.1,1.1,100) )
        y_new = Calibrator().interpolate(xs, ys, x_new)    
        x_new = x_new.detach().numpy()
        y_new = y_new.detach().numpy()
        fig, ax = plt.subplots()
        ax.plot( x_new, y_new )
        ax.scatter( xs.detach().numpy(), ys.detach().numpy() )
        plt.show()
    def forward(self, x):
        return self.interpolate( self.xs, self.ys_actual(), x )    
    
def inverse_softplus(y):
    x = np.log( np.exp(y) - 1 )
    return np.where( y > 20, y, x )

In [ ]:
class Monotonic1(torch.nn.Module):
    def __init__(self,k):
        super(Monotonic1,self).__init__()
        self.fc1 = LinearPositive(k,16)
        self.fc2 = LinearPositive(16,4)
        self.fc3 = LinearPositive(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = F.relu(y)
        y = self.fc2(y); y = F.relu(y)
        y = self.fc3(y)      # n×l×1
        p = torch.nn.Softplus()(y) + 1e-6  # Was: p=y.exp()
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 1h30min for 10,000 epochs; convergence issues" )

x, y, universe = get_data_3( flip_signs=True )

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Monotonic1(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 1_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model7_monotonic1_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True, flip_signs=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (non-linear, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model7_monotonic1_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# If the model did not converge, we often have an equal-weighted portfolio: check this is not the case
i = np.argsort( - signal.sum(axis=1) )
fig, ax = plt.subplots( figsize= (20,4) )
ax.imshow(signal.iloc[i[:50],:], aspect='auto',cmap='Blues')
ax.set_xlabel('Date')
ax.set_ylabel('Stocks')
ax.set_title("Stocks with the largest weights")
plt.show()

### Positive weights + parametrized activation function

In [ ]:
class Monotonic2(torch.nn.Module):
    def __init__(self,k):
        super(Monotonic2,self).__init__()
        self.fc1 = LinearPositive(k,16)
        self.fc2 = LinearPositive(16,4)
        self.fc3 = LinearPositive(4,1)
        self.f1 = Calibrator()
        self.f2 = Calibrator()
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = self.f1(y)
        y = self.fc2(y); y = self.f2(y)
        y = self.fc3(y)      # n×l×1
        p = torch.nn.Softplus()(y) + 1e-6  # Was: p=y.exp()
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 1h30min for 10,000 epochs; convergence issues" )

x, y, universe = get_data_3( flip_signs=True )

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Monotonic2(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 5_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model8_monotonic2_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True, flip_signs=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (non-linear, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model8_monotonic2_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# TODO: Check the activation functions learned

### Penalty for non-monotonicity

**TODO**: This does not work. Try to first estimate a nonlinear model without the penalty (that should work), and only then add the penalty.

In [ ]:
## This is just the nonlinear model above
## Differences: 
## - Also return the unnormalized score (we want the derivative, and the weights sum to 1)
## - requires_grad=True on x
## - Penalty term added to the loss function

class Monotonic3(torch.nn.Module):
    def __init__(self,k):
        super(Monotonic3,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = F.relu(y)
        y = self.fc2(y); y = F.relu(y)
        y = self.fc3(y)      # n×l×1
        p = y.exp()          # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p, y * universe    # Also return the unnormalized score

In [ ]:
LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32, requires_grad = True)   ## For the penalty, we will need the gradient wrt x
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Monotonic3(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs       = np.nan * np.zeros(N)
penalties = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    input = x[i,:,:][:,j,:]
    
    w, yhat = model( (input, universe[i,:,:][:,j,:]) )
    
    ## Penalty
    df = torch.autograd.grad(outputs = yhat.sum(), inputs = input, retain_graph = True, create_graph = True )[0]
    penalty = 1e3 * (df.clip(max=0)**2).mean()

    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR + penalty
    
    IRs[t] = IR.item()
    penalties[t] = penalty.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f} Penalty={penalties[t]:.6f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model9_monotonic3_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal, _ = model( (x,universe) )
signal = signal.detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (non-linear, signal)')
ax.text(0.02, .97, f"μ={100*res['out-of-sample'].iloc[5,:]['CAGR']:.1f}%",                  horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .90, f"σ={100*res['out-of-sample'].iloc[5,:]['Annualized Volatility']:.1f}%", horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.text(0.02, .83, f"IR={res['out-of-sample'].iloc[5,:]['Information Ratio']:.2f}",         horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
fig.savefig("plots/model9_monotonic3_wealth.pdf")
plt.show()

res['out-of-sample']

# Deep learning: optimization

In [ ]:
import cvxpy as cp
import torch 
from cvxpylayers.torch import CvxpyLayer

class OptimalPortfolio(torch.nn.Module):
    def __init__(self,k):
        super(OptimalPortfolio,self).__init__()
        self.fc = torch.nn.Linear(k,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc(x)
        
        return p, y * universe    # Also return the unnormalized score